## 1. Genetic Algorithm

In [ ]:
import numpy as np
import random

# Global variables
population = []
best_individual = None
best_fitness = float('-inf')
population_size = 100
chromosome_length = 10
crossover_rate = 0.8
mutation_rate = 0.1
generations = 100

def initialize_population(pop_size, chrom_length):
    # Modify chromosome generation based on problem encoding (binary, permutation, real-valued)
    return [generate_random_chromosome(chrom_length) for _ in range(pop_size)]

def generate_random_chromosome(length):
    # Modify based on solution representation (binary array, permutation, real values)
    return [random.random() for _ in range(length)]

def calculate_fitness(chromosome):
    # Replace with problem-specific fitness function (objective function)
    fitness = sum(chromosome)  # Example fitness - maximize sum
    return fitness

def select_parents(population, fitnesses):
    # Change selection method if needed (tournament, rank-based, roulette wheel)
    idx1, idx2 = random.sample(range(len(population)), 2)
    return population[idx1], population[idx2]

def crossover(parent1, parent2):
    # Modify crossover operator based on encoding (one-point, two-point, uniform, PMX)
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    return parent1.copy(), parent2.copy()

def mutate(chromosome):
    # Change mutation method based on encoding (bit-flip, swap, gaussian)
    mutated = chromosome.copy()
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] = random.random()  # Replace with problem-specific mutation
    return mutated

def genetic_algorithm():
    global population, best_individual, best_fitness
    
    # Initialize population
    population = initialize_population(population_size, chromosome_length)
    
    # Main evolutionary loop
    for generation in range(generations):
        # Calculate fitness for each individual
        fitnesses = [calculate_fitness(ind) for ind in population]
        
        # Track best solution
        max_fitness_idx = fitnesses.index(max(fitnesses))
        if fitnesses[max_fitness_idx] > best_fitness:
            best_fitness = fitnesses[max_fitness_idx]
            best_individual = population[max_fitness_idx].copy()
        
        # Create new population
        new_population = [best_individual]  # Elitism - keep best solution
        
        # Fill new population
        while len(new_population) < population_size:
            # Select parents
            parent1, parent2 = select_parents(population, fitnesses)
            
            # Apply crossover
            child1, child2 = crossover(parent1, parent2)
            
            # Apply mutation
            child1 = mutate(child1)
            child2 = mutate(child2)
            
            # Add to new population
            new_population.extend([child1, child2])
        
        # Replace old population (truncate if needed)
        population = new_population[:population_size]
        
        # Optional: print progress or apply early stopping
        if generation % 10 == 0:
            print(f"Generation {generation}: Best fitness = {best_fitness}")
    
    return best_individual, best_fitness

def main():
    solution, fitness = genetic_algorithm()
    print("\nFinal solution:", solution)
    print("Fitness:", fitness)

if __name__ == "__main__":
    main()

## 2. PSO

In [ ]:
import numpy as np
import random

# Global PSO parameters
num_particles = 30
num_dimensions = 5
max_iterations = 100
w = 0.7  # Inertia weight
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient
lower_bound = -10.0
upper_bound = 10.0

# Global variables for tracking
positions = None
velocities = None
pbest_positions = None
pbest_values = None
gbest_position = None
gbest_value = float('inf')  # For minimization problems

def initialize_swarm(num_particles, num_dimensions):
    # Change initialization range based on problem domain
    positions = np.random.uniform(lower_bound, upper_bound, (num_particles, num_dimensions))
    velocities = np.random.uniform(-1, 1, (num_particles, num_dimensions))
    return positions, velocities

def evaluate_fitness(position):
    # Replace with problem-specific objective function (often minimizing)
    return np.sum(position**2)  # Example: minimize sum of squares

def update_velocity(position, velocity, pbest, gbest):
    # Modify for constrained optimization or special topologies
    r1, r2 = np.random.random(num_dimensions), np.random.random(num_dimensions)
    new_velocity = (w * velocity + 
                   c1 * r1 * (pbest - position) +
                   c2 * r2 * (gbest - position))
    return new_velocity

def update_position(position, velocity):
    # Add custom constraints or boundary handling if needed
    new_position = position + velocity
    # Enforce boundaries
    new_position = np.clip(new_position, lower_bound, upper_bound)
    return new_position

def pso_algorithm():
    global positions, velocities, pbest_positions, pbest_values, gbest_position, gbest_value
    
    # Initialize swarm
    positions, velocities = initialize_swarm(num_particles, num_dimensions)
    
    # Initialize personal best positions and values
    pbest_positions = positions.copy()
    pbest_values = np.array([evaluate_fitness(p) for p in positions])
    
    # Initialize global best
    gbest_index = np.argmin(pbest_values)
    gbest_position = pbest_positions[gbest_index].copy()
    gbest_value = pbest_values[gbest_index]
    
    # Main PSO loop
    for iteration in range(max_iterations):
        # Update each particle
        for i in range(num_particles):
            # Update velocity and position
            velocities[i] = update_velocity(positions[i], velocities[i], 
                                          pbest_positions[i], gbest_position)
            positions[i] = update_position(positions[i], velocities[i])
            
            # Evaluate new position
            current_value = evaluate_fitness(positions[i])
            
            # Update personal best if improved
            if current_value < pbest_values[i]:
                pbest_values[i] = current_value
                pbest_positions[i] = positions[i].copy()
                
                # Update global best if improved
                if current_value < gbest_value:
                    gbest_value = current_value
                    gbest_position = positions[i].copy()
        
        # Optional: print progress or check convergence
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Best value = {gbest_value}")
            
    return gbest_position, gbest_value

def main():
    best_solution, best_value = pso_algorithm()
    print("\nBest solution found:", best_solution)
    print("Objective value:", best_value)

if __name__ == "__main__":
    main()

## 3. Minimax

In [ ]:
import math

# Global variables
max_player = 'X'  # Maximizing player
min_player = 'O'  # Minimizing player
max_depth = 6     # Maximum search depth

def initialize_game():
    # Create initial game state (modify based on game representation)
    return [' '] * 9  # Example: 3x3 Tic-tac-toe board

def get_possible_moves(state):
    # Return list of valid moves from current state
    return [i for i in range(len(state)) if state[i] == ' ']

def apply_move(state, move, player):
    # Apply move to state and return new state
    new_state = state.copy()
    new_state[move] = player
    return new_state

def is_terminal(state):
    # Check if state is terminal (game over)
    winner = check_winner(state)
    return winner or len(get_possible_moves(state)) == 0

def evaluate(state):
    # Heuristic evaluation function for non-terminal states
    winner = check_winner(state)
    if winner == max_player:
        return 1
    elif winner == min_player:
        return -1
    return 0  # Draw or non-terminal state

def check_winner(state):
    # Game-specific win condition check
    winning_patterns = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
    for pattern in winning_patterns:
        if state[pattern[0]] != ' ' and state[pattern[0]] == state[pattern[1]] == state[pattern[2]]:
            return state[pattern[0]]
    return None

def minimax(state, depth, is_maximizing):
    # Core minimax recursive algorithm
    if is_terminal(state) or depth == max_depth:
        return evaluate(state)
    
    if is_maximizing:
        value = -math.inf
        for move in get_possible_moves(state):
            child_state = apply_move(state, move, max_player)
            value = max(value, minimax(child_state, depth + 1, False))
        return value
    else:
        value = math.inf
        for move in get_possible_moves(state):
            child_state = apply_move(state, move, min_player)
            value = min(value, minimax(child_state, depth + 1, True))
        return value

def find_best_move(state):
    # Find best move for current player using minimax
    best_value = -math.inf
    best_move = -1
    
    for move in get_possible_moves(state):
        child_state = apply_move(state, move, max_player)
        move_value = minimax(child_state, 0, False)
        
        if move_value > best_value:
            best_value = move_value
            best_move = move
    
    return best_move

def print_board(state):
    # Display the game state visually
    for i in range(0, 9, 3):
        print(f" {state[i]} | {state[i+1]} | {state[i+2]} ")
        if i < 6:
            print("-----------")

def main():
    state = initialize_game()
    current_player = max_player
    
    while not is_terminal(state):
        print_board(state)
        
        if current_player == max_player:
            # AI turn
            move = find_best_move(state)
            state = apply_move(state, move, current_player)
            print(f"AI plays at position {move}")
        else:
            # Human turn (modify for your input handling)
            valid_moves = get_possible_moves(state)
            move = int(input(f"Enter your move {valid_moves}: "))
            state = apply_move(state, move, current_player)
        
        # Switch player
        current_player = min_player if current_player == max_player else max_player
    
    print_board(state)
    winner = check_winner(state)
    if winner:
        print(f"Player {winner} wins!")
    else:
        print("It's a draw!")

if __name__ == "__main__":
    main()

## 4. Pruning

In [ ]:
import math

# Global variables
max_player = 'X'  # Maximizing player
min_player = 'O'  # Minimizing player
max_depth = 8     # Can search deeper with pruning

def initialize_game():
    # Create initial game state (modify based on game representation)
    return [' '] * 9  # Example: 3x3 Tic-tac-toe board

def get_possible_moves(state):
    # Return list of valid moves from current state
    return [i for i in range(len(state)) if state[i] == ' ']

def order_moves(state, moves, is_maximizing):
    # Optional: Order moves for better pruning (implement heuristic ordering)
    return moves

def apply_move(state, move, player):
    # Apply move to state and return new state
    new_state = state.copy()
    new_state[move] = player
    return new_state

def is_terminal(state):
    # Check if state is terminal (game over)
    winner = check_winner(state)
    return winner or len(get_possible_moves(state)) == 0

def evaluate(state):
    # Heuristic evaluation function for non-terminal states
    winner = check_winner(state)
    if winner == max_player:
        return 1
    elif winner == min_player:
        return -1
    return 0  # Draw or non-terminal state

def check_winner(state):
    # Game-specific win condition check
    winning_patterns = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
    for pattern in winning_patterns:
        if state[pattern[0]] != ' ' and state[pattern[0]] == state[pattern[1]] == state[pattern[2]]:
            return state[pattern[0]]
    return None

def alpha_beta(state, depth, alpha, beta, is_maximizing):
    # Alpha-beta pruning algorithm with cutoff optimization
    if is_terminal(state) or depth == max_depth:
        return evaluate(state)
    
    if is_maximizing:
        value = -math.inf
        moves = order_moves(state, get_possible_moves(state), True)
        for move in moves:
            child_state = apply_move(state, move, max_player)
            value = max(value, alpha_beta(child_state, depth + 1, alpha, beta, False))
            alpha = max(alpha, value)
            if alpha >= beta:  # Beta cutoff
                break
        return value
    else:
        value = math.inf
        moves = order_moves(state, get_possible_moves(state), False)
        for move in moves:
            child_state = apply_move(state, move, min_player)
            value = min(value, alpha_beta(child_state, depth + 1, alpha, beta, True))
            beta = min(beta, value)
            if beta <= alpha:  # Alpha cutoff
                break
        return value

def find_best_move(state):
    # Find best move using alpha-beta pruning
    best_value = -math.inf
    best_move = -1
    alpha = -math.inf
    beta = math.inf
    
    for move in get_possible_moves(state):
        child_state = apply_move(state, move, max_player)
        move_value = alpha_beta(child_state, 0, alpha, beta, False)
        
        if move_value > best_value:
            best_value = move_value
            best_move = move
            
        alpha = max(alpha, best_value)
    
    return best_move

def print_board(state):
    # Display the game state visually
    for i in range(0, 9, 3):
        print(f" {state[i]} | {state[i+1]} | {state[i+2]} ")
        if i < 6:
            print("-----------")

def main():
    state = initialize_game()
    current_player = max_player
    
    while not is_terminal(state):
        print_board(state)
        
        if current_player == max_player:
            # AI turn
            move = find_best_move(state)
            state = apply_move(state, move, current_player)
            print(f"AI plays at position {move}")
        else:
            # Human turn (modify for your input handling)
            valid_moves = get_possible_moves(state)
            move = int(input(f"Enter your move {valid_moves}: "))
            state = apply_move(state, move, current_player)
        
        # Switch player
        current_player = min_player if current_player == max_player else max_player
    
    print_board(state)
    winner = check_winner(state)
    if winner:
        print(f"Player {winner} wins!")
    else:
        print("It's a draw!")

if __name__ == "__main__":
    main()